# Using keras for improving forecast accuracy
On this notebook I will compute the coeficients for using linear regression for improving the accuracy of the forecasts. I will compute many coeficcients for each site. This will allow to use more or less data depending on the date of the simulation (it is not allowed to use future data)

In [1]:
import pandas as pd
import numpy as np
import glob
import os
import json
from tqdm import tqdm_notebook

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, Callback
from keras.constraints import min_max_norm

Using TensorFlow backend.


Please modify the following path to the folder with the csv files used for training.

In [2]:
TRAIN_PATH = '/media/guillermo/Data/DrivenData/Optimize_Battery/data/train'

In [3]:
train_filepaths = sorted(glob.glob(os.path.join(TRAIN_PATH, '*.csv')))
train_filepaths = {int(os.path.splitext(os.path.basename(filepath))[0]):filepath for filepath in train_filepaths}
print('The number of train files is: %i' % len(train_filepaths))

The number of train files is: 11


In [4]:
train_dfs = {key: pd.read_csv(train_filepaths[key]) for key in tqdm_notebook(train_filepaths)}

HBox(children=(IntProgress(value=0, max=11), HTML(value='')))

Let's divide the train set into periods.

In [5]:
def get_train_samples(train_dfs):
    train_samples = {key: [] for key in train_dfs}
    for key in tqdm_notebook(train_dfs):
        df = train_dfs[key]
        _period_ids = sorted(df.period_id.unique())
        for period_id in _period_ids:
            df_period = df[df.period_id == period_id].copy()
            df_period.reset_index(drop=True, inplace=True)
            train_samples[key].append(df_period)
    return train_samples

In [6]:
train_samples = get_train_samples(train_dfs)

HBox(children=(IntProgress(value=0, max=11), HTML(value='')))

Now I want to prepare a dataset for exploration and training of the models. 
* The first feature will be the forecast. I will be using only one because I have verified that the others only contain more noise.
* The following features will be the previous values of the demand

In [7]:
def prepare_data_for_fitting(df_list, n_forecasts=10, real_column='actual_consumption', forecast_key='load_'):
    x, y = [], []
    for df in df_list:
        real_value = df.loc[n_forecasts:, 'actual_consumption'].values - df.loc[n_forecasts:, 'actual_pv'].values
        features = np.zeros((n_forecasts, len(real_value)))
        for i in range(1):
            start = n_forecasts - i - 1
            end = len(df)-i -2
            features[i] = df.loc[start:end, 'load_%02d' % (i)].values - df.loc[start:end, 'pv_%02d' % (i)].values
            
        for i in range(n_forecasts-1):
            start = n_forecasts - i - 1
            end = len(df)-i -2
            features[i+1] = df.loc[start:end, 'actual_consumption'].values - df.loc[start:end, 'actual_pv'].values
        x.append(features)
        y.append(real_value)
    x = np.concatenate(x, axis=1)
    x = np.transpose(x, axes=(1, 0))
    y = np.concatenate(y, axis=0)
    return x, y

In [8]:
def metric(demand, forecast):
    return np.mean(np.abs(demand-forecast))

## Train
I'm going to use only 3 columns for training, and I won't be using bias.
The model will be a simple linear regression.

This means that for improving the forecast I'm going to use:
* The forecast for this timestep
* The true value of the balance for the previous timestep
* The true value of the balance for the timestep before previous

In [9]:
class ModelCheckpointRAM(Callback):
    """Save the model after every epoch.
    `filepath` can contain named formatting options,
    which will be filled the value of `epoch` and
    keys in `logs` (passed in `on_epoch_end`).
    For example: if `filepath` is `weights.{epoch:02d}-{val_loss:.2f}.hdf5`,
    then the model checkpoints will be saved with the epoch number and
    the validation loss in the filename.
    # Arguments
         monitor: quantity to monitor.
        verbose: verbosity mode, 0 or 1.
        save_best_only: if `save_best_only=True`,
            the latest best model according to
            the quantity monitored will not be overwritten.
        mode: one of {auto, min, max}.
            If `save_best_only=True`, the decision
            to overwrite the current save file is made
            based on either the maximization or the
            minimization of the monitored quantity. For `val_acc`,
            this should be `max`, for `val_loss` this should
            be `min`, etc. In `auto` mode, the direction is
            automatically inferred from the name of the monitored quantity.
        save_weights_only: if True, then only the model's weights will be
            saved (`model.save_weights(filepath)`), else the full model
            is saved (`model.save(filepath)`).
        period: Interval (number of epochs) between checkpoints.
    """

    def __init__(self, monitor='val_loss', verbose=0,
                 mode='auto', period=1):
        self.monitor = monitor
        self.verbose = verbose
        self.period = period
        self.epochs_since_last_save = 0
        self.weights = None

        if mode not in ['auto', 'min', 'max']:
            warnings.warn('ModelCheckpoint mode %s is unknown, '
                          'fallback to auto mode.' % (mode),
                          RuntimeWarning)
            mode = 'auto'

        if mode == 'min':
            self.monitor_op = np.less
            self.best = np.Inf
        elif mode == 'max':
            self.monitor_op = np.greater
            self.best = -np.Inf
        else:
            if 'acc' in self.monitor or self.monitor.startswith('fmeasure'):
                self.monitor_op = np.greater
                self.best = -np.Inf
            else:
                self.monitor_op = np.less
                self.best = np.Inf

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epochs_since_last_save += 1
        if self.epochs_since_last_save >= self.period:
            self.epochs_since_last_save = 0            
            current = logs.get(self.monitor)
            if current is None:
                warnings.warn('Can save best model only with %s available, '
                              'skipping.' % (self.monitor), RuntimeWarning)
            else:
                if self.monitor_op(current, self.best):
                    if self.verbose > 0:
                        print('Epoch %05d: %s improved from %0.5f to %0.5f,'
                              ' saving model'
                              % (epoch, self.monitor, self.best, current))
                    self.best = current
                    self.weights = self.model.get_weights()
                else:
                    if self.verbose > 0:
                        print('Epoch %05d: %s did not improve' %
                              (epoch, self.monitor))

The use of the callback will garantee that the weights give the best results.

In [10]:
def multiple_train_for_all_sites(n_columns):
    df = pd.DataFrame(columns=['site_id', 'n_periods', 'timestamp', 'train_baseline', 'train_loss',
                               'val_baseline', 'val_loss', 'train_gain', 'val_gain', 'coef'])
    i = 0
    for site_id in tqdm_notebook(train_samples):
        total_periods = len(train_samples[site_id])
        for period_idx in tqdm_notebook(range(total_periods-1), total=total_periods-1, leave=False):
            n_periods = period_idx + 1

            x_train, y_train = prepare_data_for_fitting(train_samples[site_id][:n_periods])
            x_val, y_val = prepare_data_for_fitting(train_samples[site_id][n_periods:])

            x_train = x_train[:, :n_columns]
            x_val = x_val[:, :n_columns]


            model = Sequential()
            model.add(Dense(1, input_shape=(n_columns,), bias_constraint=min_max_norm(0, 0)))

            n_repeat = int(40000/len(y_train))
            if n_repeat > 0:
                x_train = np.repeat(x_train, n_repeat, axis=0)
                y_train = np.repeat(y_train, n_repeat, axis=0)

            model.compile(loss='mean_absolute_error', optimizer='Adam', metrics=[])
            callbacks = [
                EarlyStopping(patience=10, mode='min'),
                ModelCheckpointRAM(mode='min')
            ]
            ret = model.fit(x=x_train, y=y_train, batch_size=int(2**10), epochs=20000,
                            callbacks=callbacks,
                            validation_data=(x_val, y_val), verbose=0,)
            model.set_weights(callbacks[1].weights)

            train_baseline = metric(x_train[:, 0], y_train)
            val_baseline = metric(x_val[:, 0], y_val)
            train_loss = np.min(ret.history['loss'])
            val_loss = np.min(ret.history['val_loss'])
            train_gain = (train_baseline - train_loss)/train_baseline
            val_gain = (val_baseline - val_loss)/val_baseline

            coef = model.get_weights()[0][:, 0].astype(np.float64).round(4).tolist()
            timestamp = train_samples[site_id][period_idx].timestamp.values[-1]


            df.loc[i] = [site_id, n_periods, timestamp, train_baseline, train_loss, val_baseline, 
                         val_loss, train_gain, val_gain, coef]
            i += 1

    
    print('Mean train gain: %.3f' % df.mean()['train_gain'])
    print('Mean val gain: %.3f' % df.mean()['val_gain'])
    return df

df = multiple_train_for_all_sites(3)
df

HBox(children=(IntProgress(value=0, max=11), HTML(value='')))

HBox(children=(IntProgress(value=0, max=9), HTML(value='')))

HBox(children=(IntProgress(value=0, max=9), HTML(value='')))

HBox(children=(IntProgress(value=0, max=7), HTML(value='')))

HBox(children=(IntProgress(value=0, max=14), HTML(value='')))

...

HBox(children=(IntProgress(value=0, max=4), HTML(value='')))

HBox(children=(IntProgress(value=0, max=12), HTML(value='')))

HBox(children=(IntProgress(value=0, max=9), HTML(value='')))


Mean train gain: 0.022
Mean val gain: 0.021


,site_id,n_periods,timestamp,train_baseline,train_loss,val_baseline,val_loss,train_gain,val_gain,coef
0,1,1,2014-08-28 18:15:00,3107.361393,3087.252921,1753.972982,1724.834825,0.006471,0.016613,"[0.8934, 0.1117, -0.0049]"
1,1,2,2015-05-02 18:15:00,2463.298180,2422.160061,1745.815234,1720.933039,0.016700,0.014252,"[0.8955, 0.1111, -0.0064]"
2,1,3,2015-08-25 18:15:00,2258.116117,2235.627327,1731.252840,1699.808253,0.009959,0.018163,"[0.8832, 0.141, -0.0238]"
3,1,4,2015-10-24 18:15:00,2138.811485,2131.313744,1722.978716,1683.095485,0.003506,0.023148,"[0.8681, 0.1699, -0.038]"
4,1,5,2016-02-18 18:15:00,2057.764429,2039.877489,1720.859217,1680.825535,0.008692,0.023264,"[0.9069, 0.0912, 0.0018]"
5,1,6,2016-04-18 18:15:00,2018.203306,1997.163192,1695.974600,1660.449801,0.010425,0.020947,"[0.9352, 0.0545, 0.01]"
6,1,7,2017-01-31 18:15:00,1962.457028,1957.797864,1718.639680,1656.394009,0.002374,0.036218,"[0.8586, 0.1948, -0.0534]"
7,1,8,2017-04-01 18:15:00,1924.063202,1921.516326,1750.306308,1675.124389,0.001324,0.042954,"[0.8487, 0.2171, -0.066]"
8,1,9,2017-05-31 18:15:00,1919.776340,1892.206741,1615.131176,1563.079628,0.014361,0.032227,"[0.9195, 0.0897, -0.0089]"
9,10,1,2013-02-21 00:45:00,4712.436219,4597.383766,5930.306090,5769.146384,0.024415,0.027176,"[0.878, 0.18, -0.0586]"


In [11]:
df.groupby('site_id').mean()

,train_baseline,train_loss,val_baseline,val_loss,train_gain,val_gain
site_id,,,,,,
1,2205.539053,2187.212852,1717.214528,1673.838329,0.008201,0.025309
2,5174.916943,4985.983962,4962.772727,4878.917815,0.036562,0.016852
3,873.412470,859.097708,908.895920,891.264781,0.016393,0.019426
4,1029.636823,1020.891226,1030.616520,1019.204676,0.008499,0.011041
7,111.010075,108.346805,114.707263,111.615384,0.024221,0.027093
10,5517.956422,5358.925406,6183.798419,6052.327785,0.028712,0.021543
11,5018.881849,4841.809349,5108.750576,4924.851243,0.035277,0.035971
12,105.294227,101.632142,111.485112,107.061793,0.034878,0.039489
29,1335.882373,1325.017405,1352.975485,1345.448343,0.008112,0.005460


In [12]:
df.groupby('site_id').mean().mean()

train_baseline    2539.180191
train_loss        2467.859887
val_baseline      2506.305121
val_loss          2450.929443
train_gain           0.023746
val_gain             0.021827
dtype: float64

We can see that the gain is very small, about 2% of improvement.

Now I want to build a dictionary of two levels. On the first level we will find the site_id. On the second we will find the timestamp and the value will be the coefficients.

In [13]:
coef_dict = {key: {} for key in train_samples}

In [14]:
for index, row in df.iterrows():
    coef_dict[row['site_id']][row['timestamp']] = row['coef']

In [15]:
coef_dict

{1: {'2014-08-28 18:15:00': [0.8934, 0.1117, -0.0049],
  '2015-05-02 18:15:00': [0.8955, 0.1111, -0.0064],
  '2015-08-25 18:15:00': [0.8832, 0.141, -0.0238],
  '2015-10-24 18:15:00': [0.8681, 0.1699, -0.038],
  '2016-02-18 18:15:00': [0.9069, 0.0912, 0.0018],
  '2016-04-18 18:15:00': [0.9352, 0.0545, 0.01],
  '2017-01-31 18:15:00': [0.8586, 0.1948, -0.0534],
  '2017-04-01 18:15:00': [0.8487, 0.2171, -0.066],
  '2017-05-31 18:15:00': [0.9195, 0.0897, -0.0089]},
 2: {'2014-12-17 16:30:00': [0.7146, 0.3595, -0.0746],
  '2015-02-15 16:30:00': [0.8083, 0.2566, -0.0647],
  '2015-07-19 16:30:00': [0.8221, 0.2469, -0.0695],
  '2015-10-30 16:30:00': [0.7751, 0.3334, -0.1104],
  '2016-01-10 16:30:00': [0.8186, 0.2434, -0.0623]},
 3: {'2015-03-12 15:15:00': [0.8665, 0.1423, -0.0108],
  '2015-06-09 15:15:00': [0.8814, 0.1566, -0.0367],
  '2015-08-08 15:15:00': [0.8894, 0.1341, -0.0257],
  '2015-10-07 15:15:00': [0.882, 0.1518, -0.0354],
  '2016-01-10 15:15:00': [0.8847, 0.1382, -0.0253],
  '2016-0

I'm going to save this coefficients on the assets folder.

In [16]:
#with open('assets/coefs.json', 'w') as f:
#    json.dump(coef_dict, f)

The training of a model is a non-deterministic process. This could cause differences on the coefficients between different runs. Below I have saved the coeficients for site 1 on different runs.